In [1]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [60]:
# both file was downloaded from NCBI after batch entrez search with biosample list
# max number of record to process is 200

# dir for downloaded asms
asm_path= "/Users/rachel/Downloads/genome_assemblies_genome_fasta (2)/ncbi-genomes-2020-12-16/" 

# dir for xml of the downloaded asm
xml="/Users/rachel/Downloads/assembly_result (1).xml" # this file need to add a general header before process

In [63]:
asm_db = ET.parse(xml)
root = asm_db.getroot()

# put all the downloaded GCF asm into a dictionary
GCF_accn_db=dict()
for file in os.listdir(asm_path):
    GCF= "_".join(file.split("_")[0:2]) # take GCF accn from file name
    GCF_accn_db[GCF]=""
    
# match downloaded ASM file name with biosample Accession from 
for asm in root.findall("DocumentSummary"):
#     GCF = asm.find("AssemblyAccession").text # find the GCF accn in the xml
    GCF=asm.find("Synonym").find("Genbank").text # find GCA accn in the xml (when GCF record is suppressed)
    if GCF in GCF_accn_db: # check if the GCF acc is in the downloaded asm
        GCF_accn_db[GCF]=asm.find("BioSampleAccn").text # match biosample acc with GCF

    

In [65]:
# delete two summary file from dict
GCF_accn_db.pop("md5checksums.txt",None)
GCF_accn_db.pop("README.txt",None)

''

In [66]:
for file in os.listdir(asm_path):
    GCF= "_".join(file.split("_")[0:2])
    if GCF in GCF_accn_db:
        os.rename(asm_path + file, asm_path + GCF_accn_db[GCF]+".fna.gz")

In [64]:
GCF_accn_db

{'GCA_000216135.3': 'SAMN00255260',
 'GCA_000244635.3': 'SAMN00739254',
 'GCA_000216155.3': 'SAMN00255258',
 'GCA_000243935.3': 'SAMN00739244',
 'GCA_000216115.3': 'SAMN00255261',
 'GCA_015223405.1': 'SAMN14582901',
 'md5checksums.txt': '',
 'README.txt': '',
 'GCA_000244435.3': 'SAMN00739291',
 'GCA_015223395.1': 'SAMN14582900'}

In [5]:
# searched term "Leptospira" in NCBI's Assembly database - 761 records
# extract all biosamples accession from the xml

file_path="/Users/rachel/Desktop/assembly_result (2).xml"

xml=ET.parse(file_path)
xml_root=xml.getroot()

biosample_list=[]

for record in xml.findall("DocumentSummary"):
    biosample_list.append(record.find("BioSampleAccn").text)
    
    
biosample_df = pd.DataFrame(biosample_list)

biosample_df.to_csv("/Users/rachel/Desktop/ncbi_assembly_biosample_accession.txt", header=False, index=False)
